In [139]:
from langdetect import detect
import fitz
import nltk
# from nltk.probability import FreqDist

In [140]:
pdf = fitz.open("../verslag\output\CluyseDylanBP.pdf")

In [141]:
pdf.get_toc()

[[1, 'Lĳst van figuren', 7],
 [1, 'Inleiding', 9],
 [2, 'Probleemstelling', 10],
 [2, 'Onderzoeksvraag', 10],
 [2, 'Onderzoeksdoelstelling', 11],
 [2, 'Opzet van deze bachelorproef', 11],
 [1, 'Stand van zaken', 13],
 [2, 'Onderzoeken rond dyslexie', 13],
 [3, 'Centraal zicht op dyslexie', 13],
 [3, 'Fonologische dyslexie', 14],
 [3, 'Centraal zicht op de doelgroep', 14],
 [3, 'Diagnosecriteria', 15],
 [3, 'Moeilijkheden bij dyslexie', 16],
 [3, 'Aandachtspunten bij ondersteuning', 18],
 [3,
  'Bewezen voordelen van tekstvereenvoudiging bij scholieren met dyslexie',
  19],
 [2, 'Wetenschappelijke artikelen', 20],
 [3, 'Wetenschappelijke geletterdheid in Vlaanderen', 20],
 [3, 'Trends rond wetenschappelijke artikelen', 21],
 [3, 'Formaat', 23],
 [3, 'Woordenschat en vakjargon', 24],
 [3, 'Aanpak voor het lezen van wetenschappelijke artikelen', 24],
 [3, 'Conclusie', 25],
 [2, 'Tekstvereenvoudiging', 25],
 [3, 'Manuele tekstvereenvoudiging', 26],
 [3, 'Natural Language Processing', 27],


In [142]:
pdf.embfile_names()

[]

In [143]:
pdf.is_pdf

True

In [144]:
pdf.is_form_pdf

False

In [145]:
pdf.is_reflowable

False

In [146]:
pdf.needs_pass

False

In [147]:
pdf.metadata

{'format': 'PDF 1.5',
 'title': 'Scholieren met dyslexie van de derde graad middelbaar onderwijs ondersteunen bij het lezen van wetenschappelijke papers via tekstsimplificatie',
 'author': 'Dylan Cluyse',
 'subject': '',
 'keywords': '',
 'creator': 'LaTeX with hyperref',
 'producer': 'xdvipdfmx (20200315)',
 'creationDate': "D:20230306205512-00'00'",
 'modDate': '',
 'trapped': '',
 'encryption': None}

In [148]:
pdf.chapter_count

1

In [149]:
def get_pages_per_title(chosen_title):
    start = 999
    for title in pdf.get_toc():
        if title[1] == chosen_title:
            start = title[2]

        if title[2] >= start and title[1] != chosen_title:
            end = title[2]
            break

    return start, end

In [150]:
get_pages_per_title('Tekstvereenvoudiging')

(25, 26)

In [151]:
get_pages_per_title('Trends rond wetenschappelijke artikelen')

(21, 23)

In [152]:
def get_content_per_range(start, end):
    text = ""
    for i in range(start, end + 1):
        page = pdf.load_page(i)
        text += page.get_text()
    return text

In [153]:
start, end = get_pages_per_title('Trends rond wetenschappelijke artikelen')
text = get_content_per_range(start, end)

In [154]:
import re
def text_cleaning(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+\.\d+\.\d+\.', '', text) # 2.2.2 Stand van Zaken verwijderen.
    return text

In [155]:
text = text_cleaning(text)
print(text)

14 2. Stand van zaken kers (Ball, 2017; Jones e.a., 2019; Plavén-Sigray e.a., 2017). Plavén-Sigray e.a. (2017) onderzoekt de verschillende trends waarom wetenschap- pelijke artikelen alsmaar moeilijker te lezen worden. De relatie tussen de leesbaar- heid van een abstract werd vergeleken met het jaar waarin het wetenschappelijk artikel werd gepubliceerd. De Flesch-Reading-Ease of FRE score werd gebruikt om de leesgraad van een wetenschappelijk artikel te beoordelen. Om te bevesti- gen dat de relatie tussen de complexiteit van een abstract overeenstemt met die van de volledige tekstinhoud, werden er vergelijkingen gemaakt met zes verschil- lende wetenschappelijke journalen. De overeenkomst tussen de leesgraad van het abstract en de overige tekstinhoud in een wetenschappelijk artikel werd eerder be- vestigd door Dronberger en Kowitz (1975). Dat onderzoek benadrukte dat een ab- stract complexer werd geschreven, vergeleken met de rest van een wetenschappe- lijk artikel. Figuur (2.6) Afbeeld

In [156]:
language = detect(text)
language

'nl'

# Counting tokens

There are still too many tokens. We need to resort to an in-between method: extractive summarization.

In [157]:
from nltk import FreqDist
fdist = FreqDist(text)
fdist.N()

5431

## Conversion to sentences

In [158]:
text = nltk.sent_tokenize(text)
text

['14 2.',
 'Stand van zaken kers (Ball, 2017; Jones e.a., 2019; Plavén-Sigray e.a., 2017).',
 'Plavén-Sigray e.a.',
 '(2017) onderzoekt de verschillende trends waarom wetenschap- pelijke artikelen alsmaar moeilijker te lezen worden.',
 'De relatie tussen de leesbaar- heid van een abstract werd vergeleken met het jaar waarin het wetenschappelijk artikel werd gepubliceerd.',
 'De Flesch-Reading-Ease of FRE score werd gebruikt om de leesgraad van een wetenschappelijk artikel te beoordelen.',
 'Om te bevesti- gen dat de relatie tussen de complexiteit van een abstract overeenstemt met die van de volledige tekstinhoud, werden er vergelijkingen gemaakt met zes verschil- lende wetenschappelijke journalen.',
 'De overeenkomst tussen de leesgraad van het abstract en de overige tekstinhoud in een wetenschappelijk artikel werd eerder be- vestigd door Dronberger en Kowitz (1975).',
 'Dat onderzoek benadrukte dat een ab- stract complexer werd geschreven, vergeleken met de rest van een wetenschappe- 

## Extractive summarization

In [171]:
# https://github.com/wietsedv/bertje

from summarizer import Summarizer
from transformers import AutoTokenizer, AutoModel, TFAutoModel
from transformers import BertTokenizer, TFBertLMHeadModel

tokenizer = AutoTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased", revision="v1") # 2021 version
model = TFAutoModel.from_pretrained("GroNLP/bert-base-dutch-cased")

# model = TFBertLMHeadModel.from_pretrained('bert-base-cased', return_dict=True)
# tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at GroNLP/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at GroNLP/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [172]:
model

In [173]:
tokenizer

PreTrainedTokenizerFast(name_or_path='GroNLP/bert-base-dutch-cased', vocab_size=30000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

chunk creation

In [174]:
text = " ".join(text)

In [175]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dylan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [176]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))

len(chunks)

11

In [177]:
[len(tokenizer.tokenize(c)) for c in chunks]

[404, 484, 492, 443, 510, 447, 364, 451, 473, 460, 133]

In [178]:
[len(tokenizer(c).input_ids) for c in chunks]

[406, 486, 494, 445, 512, 449, 366, 453, 475, 462, 135]

In [179]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [180]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

TypeError: The current model class (TFBertModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'TFBertLMHeadModel'}

In [86]:
model = Summarizer(custom_model=model, custom_tokenizer=tokenizer)

AttributeError: 'TFBertModel' object has no attribute 'to'

In [ ]:
summary = model(text)

# Abstractive summarization + text simplification

In [ ]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import configparser, os

COMPLETIONS_MODEL = "text-davinci-003"
config = configparser.ConfigParser()
config.read('../config.ini')
openai.api_key = config['openai']['api_key']

In [181]:
amount_words_sentence = 10
amount_sentences = 4

prompt = f"""
Vereenvoudig deze tekst met deze parameters:
Zin is max {amount_words_sentence} woorden lang
Max {amount_sentences} aantal zinnen.
context: 
{text}
"""

openai.Completion.create(
    prompt=prompt,
    temperature=0,
    max_tokens=300,
    model=COMPLETIONS_MODEL
)["choices"][0]["text"].strip(" \n")

SyntaxError: incomplete input (2365192525.py, line 4)